In [1]:
import pandas as pd
import numpy as np
import pandas_profiling 
import matplotlib.pyplot as plt
import pickle 
from scipy.stats.mstats import winsorize

from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
yf.pdr_override()

%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Part 1. Data preparation

 <br>
 By Sokolov Gleb <br>
 Part of job application @ ООО ИК QBF

In [2]:
def adjsplit(price,scale,date):
    date = date +' 09:45:00'
    act_price = price[date:]
    corr_price = price[:date].iloc[:-1]/scale
    t = pd.concat([corr_price,act_price])
    return t

def setorder(tdf,col,ascending):
    dfs=[]
    #ttdf = tdf.loc[:,(slice(None),slice(None),col)].rank(axis=1,pct=True)
    for sector in sector_dict:
        rank_df= tdf.loc[:,(sector,slice(None),col)].rank(axis=1,pct=True,ascending=ascending)
        rank_df.columns = rank_df.columns.remove_unused_levels().set_levels(['rank'],level=2)

        r_rank_df = rank_df.rolling(26*2).mean()
        r_rank_df.columns = r_rank_df.columns.set_levels(['rank_r'],level=2)

        dfs.append(rank_df)
        #dfs.append(r_rank_df)
    
    ttdf = pd.concat(dfs,axis=1)
    return ttdf

## 1.1 Reading stocks data

Reading stock tickers at sectors from `./data/sectors.csv`, combining them into dictionary.
Dataframe with stock price and volume values is already preprocessed and loading from `./data/market_data_pd.pkl` file. Splits in stocks price and volume are settled with information from `./data/div_data.csv`

In [3]:
data_sectors = pd.read_csv('./data/sectors.csv').dropna()
sectors = data_sectors.INDUSTRY_SECTOR.value_counts()

In [4]:
sector_dict = {sector : data_sectors[data_sectors.INDUSTRY_SECTOR == sector].TICKER.values for sector in sectors.index}
ticker_dict = {ticker: sector for ticker,sector in zip(data_sectors.TICKER,data_sectors.INDUSTRY_SECTOR)}

In [5]:
# with open('./data/splits.pkl','rb') as f:
#     data_splits = pickle.load(f)
# columns_select = ['close','volume']
# ttdf = []
# for ticker,sector in ticker_dict.items():
#     tdf = pd.read_csv(f'./data/market_data/{ticker}.csv',index_col='dt')
#     tdf = tdf[columns_select].astype('float32')
#     tdf.columns = pd.MultiIndex.from_product([[sector],[ticker],tdf.columns])
#     try:
#         splits = data_splits[ticker]
#         print(sector,ticker,splits)
#         for split in splits:
#             tdf[(sector,ticker,'close')] = adjsplit(tdf[(sector,ticker,'close')],*split)        
#             tdf[(sector,ticker,'volume')] = adjsplit(tdf[(sector,ticker,'volume')],1/split[0],split[1])        

#     except Exception as e:
#         pass
#         #print('Error ',e)
#     ttdf.append(tdf)
        
# data = pd.concat(ttdf,axis=1,sort=True)
# data.index = pd.to_datetime(data.index)
# data = data.reindex(sorted(data.columns), axis=1)
# data.head()
# with open('./data/market_data_pd.pkl','wb') as f:
#     pickle.dump(data,f)

In [6]:
with open('./data/market_data_pd.pkl','rb') as f:
    data_all = pickle.load(f)
data_all.info() 

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 52962 entries, 2010-01-04 09:45:00 to 2018-02-05 16:00:00
Columns: 1318 entries, (Basic Materials, AA, close) to (Utilities, XEL, volume)
dtypes: float32(1318)
memory usage: 266.7 MB


In [7]:
# making sure on time range
start_day,end_day = '2010-01-01','2017-01-01'

data_all.index = (pd.to_datetime(data_all.index))
data_all = data_all.loc[start_day:end_day]
data_all.head()

In [8]:
data_all.shape

(45812, 1318)

## 1.2 Filtering data

Optional one can do data filtering. It goes under two criteria:
1. Volume
2. Price

Volume criteria depends on certain quantile value for distribution of mean values for each stock, grouped by their sectors:
$$V_{sector}^{stock} = \begin{cases}
                              V_{sector}^{stock} & \text{, }E (V_{sector}^{stock})\ge [V_{Qsector}]\\    
                              \text{NaN}    
                       \end{cases}
$$


For pricing it is percentile drop between two trading days

In [9]:
# volume_q_limit,price_pct_limit = .1,.5

# data_vol = data_all.loc[:,(slice(None),slice(None),'volume')]
# volume_limit = data_vol.mean().groupby(level=0).quantile(volume_q_limit)
# vl_rule = [((data_vol[s].mean() >= volume_limit[s]).values) for s in sectors.keys()]
# volume_rule = [i for subl in vl_rule for i in subl]

In [10]:
# data_pri = data_all_range.loc[:,(slice(None),slice(None),'close')]
# price_rule = ((np.abs(data_pri.pct_change()[1:]) >= price_pct_limit).sum() == 0)#.values

In [11]:
# ticks = price_rule.index.get_level_values(1)
# rule = ticks[(volume_rule & price_rule)].values
# tdf = data_all.loc[:,(slice(None),rule)]
# data_filtered = tdf.loc[:,(tdf.isnull().sum()<200).values].dropna()

In [12]:
# letting in only data with enough actual data in it
data_all = data_all.loc[:,(data_all.isnull().sum()<200).values]
data_all = data_all.ffill()

In [13]:
data_filtered = data_all
del data_all

In [14]:
data_filtered.shape

(45812, 1040)

## 1.3 Limiting stocks per sector

For stability of future NN model I will do sampling on the stock data per sector. So I randomly choose under N stocks in each sector.

In [15]:
n_tickers_by_sector = 20
chosen = []

for s in data_filtered.columns.levels[0]:
    temp_tickers = np.random.choice(data_filtered.loc[:,s].stack().columns,n_tickers_by_sector,replace=False)
    temp_tickers = np.unique(temp_tickers)
    chosen.append(data_filtered.loc[:,(slice(None),temp_tickers)])

data_n_by_sector = pd.concat(chosen,axis=1)

#data_n_by_sector = data_filtered

#those two started to get their Earnings data only at 2014 so...
banned_list=['CCE','MNST']
n_tickers = data_n_by_sector.columns.get_level_values(1).drop_duplicates()
try:
    for banned in banned_list:
        if banned in n_tickers:
            n_tickers = n_tickers.drop(banned)
except:
    pass


sect_lens = np.array([len(data_n_by_sector.loc[:,s].stack().columns) for s in data_n_by_sector.columns.levels[0] ])
del data_filtered
pd.DataFrame(sect_lens,index = pd.Index(list(sector_dict.keys()),name='Sector'),columns=['#']).T


Sector,"Consumer, Non-cyclical",Financial,"Consumer, Cyclical",Industrial,Technology,Energy,Communications,Utilities,Basic Materials
#,20,20,20,20,20,20,20,20,20


## 1.4 Advanced features

Little bit of feature engineering;
adding new features:
1. `r_sum` - rolling sum for difference of the price for one day;
2. `rank` - stock ranking for 'r_sum' within sector;
3. `rank_r` - rolling rank for approx. 3 month (one Q).
4. `mom` - price momentum 

In [16]:
# Market guides for sectors, can be useful
# sector_indexs = {0:'XLF',1:'XLP',2:'XLI',3:'XLE',4:'XLB',5:'XLY',6:'XLC',7:'XLK',8:'XLU',9:'SPY'}
# d = pdr.get_data_yahoo(list(sector_indexs.values())[-1], start=start_day, end=end_day)
#d.Close.to_csv('./data/INDX.csv')

In [17]:
data_n_by_sector.shape

(45812, 360)

In [18]:
for t in n_tickers:
    #data_n_by_sector[ticker_dict[t],t,'r_sum2'] = data_n_by_sector[ticker_dict[t],t,'close'].diff().rolling(26*1).sum()
    data_n_by_sector[ticker_dict[t],t,'r_sum'] = data_n_by_sector[ticker_dict[t],t,'close'].diff().rolling(26*21).sum()

# As everywhere, 26 rows in 1 day, 21 day in a month
ts=1,26*21*1,26*21*12
rs = [data_n_by_sector.loc[:,(slice(None),slice(None),'close')].shift(t) for t in ts]
ms = ((rs[1]-rs[2])/rs[2] - (rs[0]-rs[1])/rs[1])#/np.nanstd(rs[0].diff(),axis=0)
ms.columns = ms.columns.remove_unused_levels().set_levels(['mom'],level=2)

data_n_by_sector = pd.concat([data_n_by_sector,ms],axis=1)
del ms,rs
data_n_by_sector = data_n_by_sector.reindex(sorted(data_n_by_sector.columns), axis=1)
data_n_by_sector.shape

(45812, 720)

## 1.5 Adding earnings data as features

It is time to have a look into additional data we have at `./data/eps_data` - there are Earning Per Share data win announcement dates for each stock.

EPS is not actualy a great thing to do later analysys on, so lets try rearrange this information. Additional features we can create from those:
1. `EPS_L4Q` - trailing EPS for last year
2. `SUE` - suprise rating, based on earnings estimates for quater
3. `Growth` - earning growth dynamics

In [19]:
with open('./data/eps_data_list.pkl','rb') as f:
    EPSs = pickle.load(f)
    EPSs = {e:EPSs[e].loc[start_day:end_day] for e in EPSs if e in n_tickers}

In [20]:
l = [ticker_dict[ticker] for ticker in EPSs.keys()]
t = [pd.concat([df],keys = [ticker],axis=1) for ticker,df in EPSs.items()]
t = [pd.concat([df],keys = [sect],axis=1) for sect,df in zip(l,t)]

# EPS data goes beyond our pricing data, so:
EPSdf = pd.concat(t,axis=1)[:'2017']
EPSdf.dropna(axis=1, how='all',inplace=True)
del EPSs
EPSdf.head()

In [21]:
# making sure that tickers in EPS and pricing data are the same
tickers = EPSdf.columns.get_level_values(level=1).drop_duplicates()
data_full = data_n_by_sector.loc[:,(slice(None),tickers,slice(None))]

data_full = pd.concat([data_full,EPSdf],axis=1)
data_full = data_full.ffill()
data_full = data_full.reindex(sorted(data_full.columns), axis=1)

# P/E ratio thing, cliping it under 1000, pretty much random number
pe = data_full.loc[:,(slice(None),slice(None),'close')].values/data_full.loc[:,(slice(None),slice(None),'EPS_L4Q')].values
pe[pe<0] = 1000

pe_cols = data_full.loc[:,(slice(None),slice(None),'close')].columns.remove_unused_levels().set_levels(['P/E'],level=2)
pedf = pd.DataFrame(pe,index=data_full.index,columns=pe_cols)
pedf = pedf.clip(upper=1000)

# PEG ratio, as is
peg = pe / data_full.loc[:,(slice(None),slice(None),'Growth')].values
peg_cols = data_full.loc[:,(slice(None),slice(None),'close')].columns.remove_unused_levels().set_levels(['PEG'],level=2)
pegdf = pd.DataFrame(peg,index=data_full.index,columns=peg_cols)
# pegdf = pegdf.clip(upper=10)

data_full = pd.concat([pedf,pegdf,data_full],axis=1).astype('float32')
data_full = data_full.reindex(sorted(data_full.columns),axis=1)
del data_n_by_sector, pe, pedf, EPSdf,peg,pegdf

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in less
  # This is added back by InteractiveShellApp.init_path()
/usr/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in true_divide


## 1.6 Simple ranking

The idea is to rank each of predictive features across market within their sectors, then use those ranks as inputs for deep neural ranking net, based on listwise strategy. To improve individual rankings there are volatility measure and feature for boosting scores with momentum data.

In [22]:
# Getting some rating of volatility 
dfs={}
for s in sector_dict:
    vol = data_full.loc[:,(s,slice(None),'volume')][:]
    #vol[vol<0]=np.nan
    vol_std = vol.diff().rolling(26*21).std()
    dfs[s] = vol_std.mean(axis=1)
volatility_rating = pd.concat(dfs,axis=1)
volatility_rating = volatility_rating.div(volatility_rating.max(axis=1), axis=0)#.mean()
#volatility_rating.plot()

In [29]:
dfs = []

# Tensor for boosting ranking with good predictive feature, Momentum
# boost = 2*(data_full.loc[:,(slice(None),slice(None),'mom')].rank(axis=1,pct=True,ascending=False))-1
# boost = (1/(1-boost.abs())**9)
# boost = (1+boost.div(boost.max(axis=1), axis=0))

# ranking features
for c,asc in zip(['mom','P/E','PEG','r_sum'],[True,True,True,True]):
    df = 2*setorder(data_full,c,ascending=asc)-1
    #df = df*boost.values
#     for s in sector_dict:
#         df[s] = df[s].div(volatility_rating[s],axis=0)
    df.columns = df.columns.remove_unused_levels().set_levels([c+'_rank'],level=2)
    dfs.append(df)
ranks = pd.concat(dfs,axis=1)
del dfs
with open('./data/ranks.pkl','wb') as f:
    pickle.dump(ranks,f)

In [ ]:
rank = ranks.groupby(level=1,axis=1).mean()
#w = (.3,.3,.3,.1)
#rank = t.dropna().groupby(level=1,axis=1).apply(lambda y: (y.iloc[:,0]*w[0] + w[1]*y.iloc[:,1] + y.iloc[:,2]*w[2] + y.iloc[:,3]*w[3]))
rank_df = pd.DataFrame(rank.values,index=rank.index,columns=ranks.columns)
del ranks,rank

data_r = pd.concat([data_full,rank_df],axis=1).astype('float32')
data_r = data_r.reindex(sorted(data_r.columns), axis=1)

del rank_df

In [ ]:
with open('./data/market.pkl','wb') as f:
    pickle.dump(data_r,f)

## 1.7 Visualising

In [ ]:
data_full.Technology.loc[:,(slice(None),'rank')]['2013'].plot(figsize=(15,7));

In [ ]:
data_full.Technology.ADSK.close['2013'].diff().sum()#.plot(figsize=(20,10))

In [ ]:
data.loc[:,('Industrial',slice(None),'close')]['2013-01':'2013-04'].diff().sum().sort_values(ascending=False)

In [ ]:
data.Industrial.loc[:,(slice(None),'rank_r')]['2013-01':'2013-04'].mean().sort_values()

In [ ]:
# ar = data_full.Financial.loc[:,(slice(None),'rank')].values.T
# ms = ~np.isnan(ar)

# f,a = plt.subplots(1,1,figsize=(10,7))

# [plt.plot(a[m]) for a,m in zip(ar,ms)];

In [ ]:
se,ti = 'Financial','AMG'
date1,date2 = '2016','2016'

fig, axes = plt.subplots(1,1, figsize=(15,7), sharex=True)

e = EPSdf[se][ti].dropna()[date1:date2].index
b = e - pd.Timedelta(hours=24*5)
time_pairs = [(bi,ei) for ei,bi in zip(e,b)]

data.Technology.loc[:,(slice(None),['rank_r'])][date1:date2].plot(figsize=(20,10),ax=axes);

[axes.axvspan(*p,color='g',alpha=.2) for p in time_pairs];
#data_full[se][ti][date1:date2]['rank_r'].dropna().plot(subplots = True,ax=axes);
